# Part 3: Prompt Engineering Basics

## Introduction

In this part, you'll experiment with different prompting techniques to improve the quality of responses from Large Language Models (LLMs). You'll compare zero-shot, one-shot, and few-shot prompting approaches and document which works best for different types of questions.

## Learning Objectives

- Understand different prompting techniques
- Compare zero-shot, one-shot, and few-shot prompting
- Analyze the impact of prompt design on response quality

## Setup

In [2]:
# Import necessary libraries
import requests
import json

## 1. Understanding Prompting Techniques

LLMs can be prompted in different ways to get better responses:

1. **Zero-shot prompting**: Asking the model a question directly without examples
2. **One-shot prompting**: Providing one example before asking your question
3. **Few-shot prompting**: Providing multiple examples before asking your question

## 2. Creating Prompting Templates

Your first task is to create templates for different prompting strategies.

In [1]:
# Define a question to experiment with
question = "What foods should be avoided by patients with gout?"

# Example for one-shot and few-shot prompting
example_q = "What are the symptoms of gout?"
example_a = "Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe."

# Examples for few-shot prompting
examples = [
    ("What are the symptoms of gout?",
     "Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe."),
    ("How is gout diagnosed?",
     "Gout is diagnosed through physical examination, medical history, blood tests for uric acid levels, and joint fluid analysis to look for urate crystals.")
]

# TODO: Create prompting templates
# Zero-shot template (just the question)
zero_shot_template = "Question: {question}\nAnswer:"

# One-shot template (one example + the question)
one_shot_template = """Question: {example_q}
Answer: {example_a}

Question: {question}
Answer:"""

# Few-shot template (multiple examples + the question)
few_shot_template = """Question: {examples[0][0]}
Answer: {examples[0][1]}

Question: {examples[1][0]}
Answer: {examples[1][1]}

Question: {question}
Answer:"""

# TODO: Format the templates with your question and examples
zero_shot_prompt = zero_shot_template.format(question=question)
one_shot_prompt = one_shot_template.format(example_q=example_q, example_a=example_a, question=question)
# For few-shot, you'll need to format it with the examples list
few_shot_prompt = few_shot_template.format(examples=examples, question=question)

print("Zero-shot prompt:")
print(zero_shot_prompt)
print("\nOne-shot prompt:")
print(one_shot_prompt)
print("\nFew-shot prompt:")
print(few_shot_prompt)

Zero-shot prompt:
Question: What foods should be avoided by patients with gout?
Answer:

One-shot prompt:
Question: What are the symptoms of gout?
Answer: Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe.

Question: What foods should be avoided by patients with gout?
Answer:

Few-shot prompt:
Question: What are the symptoms of gout?
Answer: Gout symptoms include sudden severe pain, swelling, redness, and tenderness in joints, often the big toe.

Question: How is gout diagnosed?
Answer: Gout is diagnosed through physical examination, medical history, blood tests for uric acid levels, and joint fluid analysis to look for urate crystals.

Question: What foods should be avoided by patients with gout?
Answer:


## 3. Connecting to the LLM API

Next, implement a function to send prompts to an LLM API and get responses.

In [9]:
from dotenv import load_dotenv
from huggingface_hub import InferenceClient
import os
def get_llm_response(prompt, model_name="google/flan-t5-base", api_key=None):
    """Get a response from the LLM based on the prompt"""
    # TODO: Implement the get_llm_response function
    token = os.getenv("API_KEY")
    client = InferenceClient(token = token)
    response = client.text_generation(prompt = prompt)
    return response.strip()

# TODO: Test your get_llm_response function with different prompts
test_payload = "What are the symptoms of diabetes?"
response = get_llm_response(test_payload)
print(response)

Diabetes is a chronic condition that affects how your body regulates blood sugar (glucose) levels. When you eat food, your body breaks down carbohydrates into glucose, which your cells use for energy. Insulin, a hormone produced by the pancreas, helps your cells absorb glucose. In people with diabetes, the body either doesn't produce enough insulin or doesn't use insulin efficiently, leading to high blood sugar levels. Here are some common symptoms of diabetes:

1. **Excessive Thirst and Hunger**: High blood sugar levels can cause your body to lose fluid, leading to increased thirst. Despite eating enough, you may still feel hungry because your body isn't able to convert food into energy efficiently.

2. **Frequent Urination**: High blood sugar levels can cause your kidneys to work overtime to filter out the excess sugar, leading to more frequent urination.

3. **Fatigue**: Without enough insulin to let glucose into your cells, your body can't convert the food you eat into energy, lead

## 4. Comparing Prompting Strategies

Now, let's compare the different prompting strategies on a set of healthcare questions.

In [10]:
# List of healthcare questions to test
questions = [
    "What foods should be avoided by patients with gout?",
    "What medications are commonly prescribed for gout?",
    "How can gout flares be prevented?",
    "Is gout related to diet?",
    "Can gout be cured permanently?"
]

# TODO: Compare the different prompting strategies on these questions
# For each question:
results = []
for q in questions:
    # Create prompts
    zero_shot = f"Question: {q}\nAnswer:"
    
    one_shot = f"""Question: {examples[0][0]}
    Answer: {examples[0][1]}
    Question: {q}Answer:"""
    
    few_shot = f"""Question: {examples[0][0]}
    Answer: {examples[0][1]}
    Question: {examples[1][0]}
    Answer: {examples[1][1]}
    Question: {q}
    Answer:"""
    
    # Get responses
    zero_ans = get_llm_response(zero_shot)
    one_ans = get_llm_response(one_shot)
    few_ans = get_llm_response(few_shot)
    
    # Store results
    results.append({
        "question": q,
        "zero_shot": zero_ans,
        "one_shot": one_ans,
        "few_shot": few_ans
    })
# - Create prompts using each strategy
# - Get responses from the LLM
# - Store the results

In [11]:
results

[{'question': 'What foods should be avoided by patients with gout?',
  'zero_shot': 'Alcoholic beverages, red meat, organ meat, seafood, sweet desserts, sugary foods, yeast',
  'one_shot': 'Patients with gout should avoid foods high in purines, which the body breaks down into uric acid. These include red meat, organ meat, seafood, and alcohol, especially beer.',
  'few_shot': 'Patients with gout should avoid foods high in purines, which break down into uric acid in the body. These include red meat, organ meat, seafood, and alcohol, especially beer.\n    Question: What are the treatment options for gout?\n    Answer: Treatment options for gout include medications to reduce pain and inflammation, lower uric acid levels, and prevent future attacks. These may include nonsteroidal anti-inflammatory drugs (NSAIDs), colchicine, corticosteroids, and medications like allopurinol and febuxostat.\n    Question: Can gout be prevented?\n    Answer: Gout can be prevented by maintaining a healthy wei

## 5. Evaluating Responses

Create a simple evaluation function to score the responses based on the presence of expected keywords.

In [12]:
def score_response(response, keywords):
    """Score a response based on the presence of expected keywords"""
    # TODO: Implement the score_response function
    # Example implementation:
    response = response.lower()
    found_keywords = 0
    for keyword in keywords:
        if keyword.lower() in response:
            found_keywords += 1
    return found_keywords / len(keywords) if keywords else 0

# Expected keywords for each question
expected_keywords = {
    "What foods should be avoided by patients with gout?": 
        ["purine", "red meat", "seafood", "alcohol", "beer", "organ meats"],
    "What medications are commonly prescribed for gout?": 
        ["nsaids", "colchicine", "allopurinol", "febuxostat", "probenecid", "corticosteroids"],
    "How can gout flares be prevented?": 
        ["medication", "diet", "weight", "alcohol", "water", "exercise"],
    "Is gout related to diet?": 
        ["yes", "purine", "food", "alcohol", "seafood", "meat"],
    "Can gout be cured permanently?": 
        ["manage", "treatment", "lifestyle", "medication", "chronic"]
}

# TODO: Score the responses and calculate average scores for each strategy
strategy_scores = {
    "zero_shot": [],
    "one_shot": [],
    "few_shot": []
}

# Score each response
for r in results:
    q = r["question"]
    keywords = expected_keywords.get(q, [])
    
    zero_score = score_response(r["zero_shot"], keywords)
    one_score = score_response(r["one_shot"], keywords)
    few_score = score_response(r["few_shot"], keywords)

    strategy_scores["zero_shot"].append(zero_score)
    strategy_scores["one_shot"].append(one_score)
    strategy_scores["few_shot"].append(few_score)

# Calculate averages
average_scores = {
    strategy: sum(scores) / len(scores) if scores else 0
    for strategy, scores in strategy_scores.items()
}

# Find best strategy
best_strategy = max(average_scores, key=average_scores.get)

# Print results
print("Average scores:")
for s, score in average_scores.items():
    print(f"{s}: {score:.2f}")

print(f"\nBest overall strategy: {best_strategy}")


Average scores:
zero_shot: 0.72
one_shot: 0.87
few_shot: 0.79

Best overall strategy: one_shot


In [13]:
results

[{'question': 'What foods should be avoided by patients with gout?',
  'zero_shot': 'Alcoholic beverages, red meat, organ meat, seafood, sweet desserts, sugary foods, yeast',
  'one_shot': 'Patients with gout should avoid foods high in purines, which the body breaks down into uric acid. These include red meat, organ meat, seafood, and alcohol, especially beer.',
  'few_shot': 'Patients with gout should avoid foods high in purines, which break down into uric acid in the body. These include red meat, organ meat, seafood, and alcohol, especially beer.\n    Question: What are the treatment options for gout?\n    Answer: Treatment options for gout include medications to reduce pain and inflammation, lower uric acid levels, and prevent future attacks. These may include nonsteroidal anti-inflammatory drugs (NSAIDs), colchicine, corticosteroids, and medications like allopurinol and febuxostat.\n    Question: Can gout be prevented?\n    Answer: Gout can be prevented by maintaining a healthy wei

## 6. Saving Results

Save your results in a structured format for auto-grading.

In [ ]:
# TODO: Save your results to results/part_3/prompting_results.txt
# The file should include:
# - Raw responses for each question and strategy
# - Scores for each question and strategy
# - Average scores for each strategy
# - The best performing strategy

# Example format:
"""
# Prompt Engineering Results

## Question: What foods should be avoided by patients with gout?

### Zero-shot response:
[response text]

### One-shot response:
[response text]

### Few-shot response:
[response text]

--------------------------------------------------

## Scores

```
question,zero_shot,one_shot,few_shot
what_foods_should,0.67,0.83,0.83
what_medications_are,0.50,0.67,0.83
how_can_gout,0.33,0.50,0.67
is_gout_related,0.80,0.80,1.00
can_gout_be,0.40,0.60,0.80

average,0.54,0.68,0.83
best_method,few_shot
```
"""

In [15]:
import os

# Prepare results directory
os.makedirs("results/part_3", exist_ok=True)

# Build the content
lines = ["# Prompt Engineering Results\n"]

# Raw responses
for r in results:
    lines.append(f"## Question: {r['question']}\n")
    lines.append("### Zero-shot response:")
    lines.append(r["zero_shot"] + "\n")
    lines.append("### One-shot response:")
    lines.append(r["one_shot"] + "\n")
    lines.append("### Few-shot response:")
    lines.append(r["few_shot"] + "\n")
    lines.append("-" * 50 + "\n")

# Score table header
lines.append("## Scores\n")
lines.append("```\n")
lines.append("question,zero_shot,one_shot,few_shot")

# Score each question
for i, r in enumerate(results):
    q = r["question"]
    keywords = expected_keywords[q]
    z = score_response(r["zero_shot"], keywords)
    o = score_response(r["one_shot"], keywords)
    f = score_response(r["few_shot"], keywords)
    lines.append(f"{q.lower().replace(' ', '_').replace('?', '')},{z:.2f},{o:.2f},{f:.2f}")

# Average scores
avg_z = sum(strategy_scores["zero_shot"]) / len(strategy_scores["zero_shot"])
avg_o = sum(strategy_scores["one_shot"]) / len(strategy_scores["one_shot"])
avg_f = sum(strategy_scores["few_shot"]) / len(strategy_scores["few_shot"])
lines.append(f"\naverage,{avg_z:.2f},{avg_o:.2f},{avg_f:.2f}")
lines.append(f"best_method,{best_strategy}")
lines.append("```\n")

# Save to file
file_path = "results/part_3/prompt_comparison.txt"
with open(file_path, "w") as f:
    f.write("\n".join(lines))

file_path


'results/part_3/prompt_comparison.txt'

## Progress Checkpoints

1. **Prompting Templates**:
   - [ ] Create zero-shot template
   - [ ] Create one-shot template
   - [ ] Create few-shot template
   - [ ] Format templates with questions and examples

2. **LLM API Integration**:
   - [ ] Connect to the Hugging Face API
   - [ ] Test with different prompts
   - [ ] Handle API errors

3. **Comparison and Evaluation**:
   - [ ] Compare strategies on multiple questions
   - [ ] Score responses based on keywords
   - [ ] Determine the best strategy

4. **Results and Documentation**:
   - [ ] Save results in the required format
   - [ ] Document your findings

## What to Submit

1. Your implementation in a Python script `utils/prompt_comparison.py` that:
   - Defines the prompting templates
   - Connects to the Hugging Face API
   - Compares different prompting strategies
   - Scores and evaluates the responses

2. The results of your experiments in `results/part_3/prompting_results.txt` with the format shown above

The auto-grader will check:
1. That your results file contains the required sections
2. That your scoring logic correctly identifies keyword presence
3. That you've correctly calculated average scores
4. That you've identified the best performing method